# Human detection on __Micro Controllers__

## This file is using for Human detection.
### Datasets: https://www.kaggle.com/datasets/constantinwerner/human-detection-dataset -- Authors: KONSTANTIN VERNER

$$ $$
$$ File's  Author: NhutNM $$
$$ File's  Author-email: nhut0550.1109@gmail.com $$
$$ File's  Author-github: github.com/mnhut0550 $$
$$ $$

##### *This file is use Sparse_Categorical_Crossentropy as loss function.
##### *FULL QUANT edition.

## import library

In [1]:
import tensorflow as tf

import numpy as np
import tempfile
import zipfile
import os

## Create class representative_dataset for converter

In [2]:
class representive_dataset:
    def __init__(self, img_list:list, batch_size=64, dtype=tf.int8):
        self.data = np.array(img_list)
        self.batch = batch_size
        self.dtype = dtype
        self.img = tf.data.Dataset.from_tensor_slices(self.data).batch(1)

    def __call__(self):
        for data in self.img.take(self.batch):
            yield [data]

## select base model and model's input_size

In [3]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input

baseModel = MobileNet(input_shape=(96,96,3), alpha = 0.25, include_top=False, weights='imagenet')

## load data

In [4]:
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from imutils import paths

imagePath = list(paths.list_files('../data/'))
data = []
labels = []

In [5]:
for image in imagePath:
    label = int(image.split(os.path.sep)[-2].split('/')[-1])
     
    img = load_img(image,target_size = (96,96))
    img = img_to_array(img)
    img = preprocess_input(img)
    img.astype(np.float32)
     
    data.append(img)
    labels.append(label)

## convert datatype

In [6]:
data = np.array(data, dtype=np.float32)
labels = np.array(labels)

## split data

In [7]:
from sklearn.model_selection import train_test_split

(trainX, valX, trainY, valY) = train_test_split(data, labels,test_size=0.2, stratify=labels, random_state=42)
testX = valX
testY = valY

## create model

### import layer module & model wrapper module

In [8]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

### create new model base on base_model

In [9]:
inp = baseModel.input

x = baseModel.output
x = Dropout(0.1)(x)
x = AveragePooling2D()(x)
x = Conv2D(2,use_bias= False, kernel_size=x.get_shape().as_list()[1], activation='relu')(x)
x = Flatten()(x)
x = Dense(2)(x)

out = x

In [10]:
model = Model(inputs=inp, outputs= out)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 48, 48, 8)         216       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 48, 48, 8)         32        
_________________________________________________________________
conv1_relu (ReLU)            (None, 48, 48, 8)         0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 48, 48, 8)         72        
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 48, 48, 8)         32        
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 48, 48, 8)         0     

## compile model

In [12]:
from tensorflow.keras.optimizers import Adam, SGD 
from tensorflow.keras.losses import SparseCategoricalCrossentropy as CC

opt = Adam(1e-4)
model.compile(optimizer=opt, loss = CC(from_logits=True), metrics=['accuracy'])

## training

In [13]:
from tensorflow.keras.callbacks import EarlyStopping

ES = EarlyStopping(patience=2, monitor='val_accuracy')

model.fit(trainX, trainY, epochs=30, validation_data=(valX, valY), callbacks=[ES], use_multiprocessing=True)

Epoch 1/30
23/23 [==============================] - 6s 113ms/step - loss: 0.6526 - accuracy: 0.5992 - val_loss: 0.6378 - val_accuracy: 0.5946
Epoch 2/30
23/23 [==============================] - 2s 75ms/step - loss: 0.5832 - accuracy: 0.6617 - val_loss: 0.5857 - val_accuracy: 0.6865
Epoch 3/30
23/23 [==============================] - 2s 75ms/step - loss: 0.5222 - accuracy: 0.7269 - val_loss: 0.5424 - val_accuracy: 0.7405
Epoch 4/30
23/23 [==============================] - 2s 73ms/step - loss: 0.4727 - accuracy: 0.7840 - val_loss: 0.5013 - val_accuracy: 0.7568
Epoch 5/30
23/23 [==============================] - 2s 72ms/step - loss: 0.4062 - accuracy: 0.8410 - val_loss: 0.4670 - val_accuracy: 0.8108
Epoch 6/30
23/23 [==============================] - 2s 72ms/step - loss: 0.3510 - accuracy: 0.8859 - val_loss: 0.4378 - val_accuracy: 0.8216
Epoch 7/30
23/23 [==============================] - 2s 73ms/step - loss: 0.3052 - accuracy: 0.9035 - val_loss: 0.4141 - val_accuracy: 0.8270
Epoch 8/30
2

## evaluate model

In [14]:
loss, acc = model.evaluate(testX, testY)
acc

6/6 [==============================] - 0s 17ms/step - loss: 0.3709 - accuracy: 0.8378


0.837837815284729

## save original model

In [15]:
# model.save('model.h5')

# Unfreeze layer

In [16]:
for layer in model.layers:
    if layer.trainable == False:
        print(layer.name)
        layer.trainable = True

# Pruning

## set pruning infor

In [17]:
import tensorflow_model_optimization as tfmot

prune_low_mag = tfmot.sparsity.keras.prune_low_magnitude

pruning_params = {"pruning_schedule": tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.3, final_sparsity=0.9, begin_step=0, end_step=20)}

callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]

def apply_pruning_to_unsupport_layer(layer):
    try:
        return prune_low_mag(layer, **pruning_params)
    except:
        print(layer.name)
        return layer

pruned_model = tf.keras.models.clone_model(
    model,
    clone_function=apply_pruning_to_unsupport_layer
)


## compile pruning model

In [18]:

opt = Adam(learning_rate=1e-4)
pruned_model.compile(optimizer=opt, loss =  CC(from_logits=True), metrics=['accuracy'])

## train pruning model

In [19]:
pruned_model.fit(trainX, trainY, epochs=4, validation_data=(valX, valY), callbacks=callbacks)

Epoch 1/4
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
23/23 [==============================] - 15s 135ms/step - loss: 0.1804 - accuracy: 0.9524 - val_loss: 0.3600 - val_accuracy: 0.8432
Epoch 2/4
23/23 [==============================] - 2s 106ms/step - loss: 0.1448 - accuracy: 0.9620 - val_loss: 0.3488 - val_accuracy: 0.8432
Epoch 3/4
23/23 [==============================] - 2s 105ms/step - loss: 0.1211 - accuracy: 0.9755 - val_loss: 0.3476 - val_accuracy: 0.8432
Epoch 4/4
23/23 [==============================] - 2s 105ms/step - loss: 0.1032 - accuracy: 0.9851 - val_loss: 0.3439 - val_accuracy: 0.8486


## evaluate pruned model

In [20]:
loss, acc = pruned_model.evaluate(testX, testY)
acc

6/6 [==============================] - 0s 19ms/step - loss: 0.3439 - accuracy: 0.8486


0.8486486673355103

## strip prune infor

In [21]:
stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

## save pruned model

In [22]:
# stripped_pruned_model.save('PRUNED_MODEL/model.h5')

# Clustering

## set clustering infor

In [23]:
from tensorflow_model_optimization.python.core.clustering.keras.experimental import (cluster,)

cluster_weight = tfmot.clustering.keras.cluster_weights

centroidInit = tfmot.clustering.keras.CentroidInitialization

cluster_weight = cluster.cluster_weights

clustering_params = {
    "number_of_clusters": 64,
    "cluster_centroids_init": centroidInit.RANDOM,
    "preserve_sparsity": True
}

#### If model has no unsupport layer, use this function
# sparsity_clustered_model = cluster_weight(stripped_pruned_model, **clustering_params) 

#### else use this way!

def apply_cluster_to_unsupport_layer(layer):
    try:
        return cluster_weight(layer, **clustering_params)
    except:
        print(layer.name)
        return layer

sparsity_clustered_model = tf.keras.models.clone_model(
    stripped_pruned_model,
    clone_function=apply_cluster_to_unsupport_layer
)

conv_dw_1
conv_dw_2
conv_dw_3
conv_dw_4
conv_dw_5
conv_dw_6
conv_dw_7
conv_dw_8
conv_dw_9
conv_dw_10
conv_dw_11
conv_dw_12
conv_dw_13


## compile clustering model

In [24]:
sparsity_clustered_model.compile(optimizer=Adam(learning_rate=1e-4),
                        loss =  CC(from_logits=True),
                        metrics = ['accuracy'])

## training clustering model

In [25]:
sparsity_clustered_model.fit(trainX, trainY, epochs=3, validation_data=(valX, valY))

Epoch 1/3
23/23 [==============================] - 7s 178ms/step - loss: 0.6427 - accuracy: 0.6318 - val_loss: 0.5990 - val_accuracy: 0.6757
Epoch 2/3
23/23 [==============================] - 3s 144ms/step - loss: 0.5525 - accuracy: 0.7514 - val_loss: 0.5337 - val_accuracy: 0.7405
Epoch 3/3
23/23 [==============================] - 3s 150ms/step - loss: 0.4548 - accuracy: 0.7894 - val_loss: 0.4245 - val_accuracy: 0.7892


## evaluate clustered model

In [26]:
loss,acc = sparsity_clustered_model.evaluate(testX, testY)
acc

6/6 [==============================] - 0s 49ms/step - loss: 0.4245 - accuracy: 0.7892


0.7891891598701477

## strip cluster infor from model

In [27]:
stripped_sparsity_clustered_model = tfmot.clustering.keras.strip_clustering(sparsity_clustered_model)

## save clustered model

In [28]:
# stripped_sparsity_clustered_model.save('SPARSITY_CLUSTERED/model.h5')

# PCQAT

## apply PCQAT to pruned & clustered model

In [29]:
QAT_model = tfmot.quantization.keras.quantize_annotate_model(stripped_sparsity_clustered_model)

##### Warmning for custom Activaton Layer & ReLU6
pcqat = tfmot.quantization.keras.quantize_apply(QAT_model,
                                    tfmot.experimental.combine.Default8BitClusterPreserveQuantizeScheme(preserve_sparsity=True))


In [30]:
pcqat.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
quantize_layer (QuantizeLaye (None, 96, 96, 3)         3         
_________________________________________________________________
quant_conv1 (QuantizeWrapper (None, 48, 48, 8)         704       
_________________________________________________________________
quant_conv1_bn (QuantizeWrap (None, 48, 48, 8)         33        
_________________________________________________________________
quant_conv1_relu (QuantizeWr (None, 48, 48, 8)         3         
_________________________________________________________________
quant_conv_dw_1 (QuantizeWra (None, 48, 48, 8)         75        
_________________________________________________________________
quant_conv_dw_1_bn (Quantize (None, 48, 48, 8)         33    

## compile PCQAT model

In [31]:
pcqat.compile(optimizer=Adam(learning_rate=3e-5),
                loss = CC(),
                metrics = ['accuracy'])

## training PCQAT model

In [32]:
pcqat.fit(trainX, trainY, epochs=2, validation_data=(valX, valY))

Epoch 1/2


23/23 [==============================] - 11s 255ms/step - loss: 0.5954 - accuracy: 0.9062 - val_loss: 0.6566 - val_accuracy: 0.7838
Epoch 2/2
23/23 [==============================] - 5s 209ms/step - loss: 0.5608 - accuracy: 0.9158 - val_loss: 0.7329 - val_accuracy: 0.8054


In [33]:
loss,acc = pcqat.evaluate(testX,testY)
acc

6/6 [==============================] - 0s 41ms/step - loss: 0.7329 - accuracy: 0.8054


0.8054053783416748

## save PCQAT model

In [34]:
# pcqat.save('PCQAT/model.h5')

# Convert PCQAT to TFLITE

## Convert to INT8-TFLITE (include input layer)

In [35]:
gen = representive_dataset(trainX, dtype=tf.int8)

In [36]:
converter = tf.lite.TFLiteConverter.from_keras_model(pcqat)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_LATENCY]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = gen
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\NC\AppData\Local\Temp\tmp4ljk2b8x\assets


INFO:tensorflow:Assets written to: C:\Users\NC\AppData\Local\Temp\tmp4ljk2b8x\assets


## save TFLITE model

In [37]:
with open('model_pcqat.tflite',"wb") as f:
    f.write(tflite_model)